In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf


In [ ]:
import pandas as pd
import numpy as np
import re

# Read the CSV file
data = pd.read_csv('twitter_sentiment_data.csv', sep=',', dtype={'tweetid': str})

# Print the DataFrame to make sure it's populated with data
print(data.head())

def preprocess_text(text):
    # Check if the input is a string
    if not isinstance(text, str):
        return ''

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and numbers
    text = re.sub(r'\W+|\d+', ' ', text)

    # Convert to lowercase
    text = text.lower()

    return text

# Apply the preprocessing function to the 'message' column
data['cleaned_text'] = data['message'].apply(preprocess_text)

# Print the DataFrame to make sure the 'cleaned_text' column is created
print(data.head())

# Save the preprocessed data to a CSV file
data.to_csv('preprocessed_twitter_sentiment_data.csv', index=False)

# Sample data
sample_size = 3  # Adjust this number to display more or fewer samples

# Check if the DataFrame is not empty before sampling
if not data.empty:
    sample_data = data.sample(sample_size)
    print(sample_data[['message', 'cleaned_text', 'sentiment']])
else:
    print("The DataFrame is empty.")




In [ ]:
from gensim.models import Word2Vec

# Read the CSV file
df = pd.read_csv('cleaned_twitter_sentiment_data.csv',encoding='iso-8859-1')

# Prepare the dataset
dataset = df['cleaned_text'].apply(lambda x: x.split()).tolist()



# Train the Word2Vec model
model = Word2Vec(dataset, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Convert the preprocessed tweets into word vectors
tweet_vectors = []

for tweet in dataset:
    tweet_vector = []
    for word in tweet:
        if word in model.wv:
            tweet_vector.append(model.wv[word])
    tweet_vectors.append(tweet_vector)

# Print the first tweet and its corresponding word vectors
print('Tweet:', df.loc[0, 'cleaned_text'])
print('Word Vectors:', tweet_vectors[0])



In [ ]:
# Function to create sentence embeddings
def sentence_embedding(words, model, vector_size):
    valid_words = [word for word in words if word in model.wv]
    if valid_words:
        embeddings = np.array([model.wv[word] for word in valid_words])
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(vector_size)

# Create sentence embeddings for each tweet
vector_size = 100
data['embedding'] = data['cleaned_text'].apply(lambda x: sentence_embedding(x.split(), model, vector_size))

# Prepare the features and target
X = np.stack(data['embedding'].values)
y = data['sentiment'].values


In [ ]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate the model
print('Logistic Regression')
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


Logistic Regression
              precision    recall  f1-score   support

          -1       0.58      0.18      0.28       784
           0       0.62      0.40      0.48      1582
           1       0.68      0.88      0.76      4514
           2       0.74      0.65      0.69      1909

    accuracy                           0.68      8789
   macro avg       0.65      0.53      0.55      8789
weighted avg       0.67      0.68      0.65      8789



In [ ]:
from sklearn.svm import SVC
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an SVM model
clf = SVC()
clf.fit(X_train, y_train)

# Evaluate the model
print('SVM')
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


SVM
              precision    recall  f1-score   support

          -1       0.79      0.06      0.11       784
           0       0.72      0.28      0.41      1582
           1       0.64      0.92      0.76      4514
           2       0.75      0.66      0.70      1909

    accuracy                           0.67      8789
   macro avg       0.73      0.48      0.49      8789
weighted avg       0.69      0.67      0.62      8789



In [ ]:
from sklearn.naive_bayes import GaussianNB

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Gaussian Naive Bayes model
clf = GaussianNB()
clf.fit(X_train, y_train)

# Evaluate the model
print('Gaussian Naive Bayes')
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))



Gaussian Naive Bayes
              precision    recall  f1-score   support

          -1       0.23      0.59      0.33       784
           0       0.49      0.35      0.41      1582
           1       0.76      0.52      0.62      4514
           2       0.57      0.74      0.65      1909

    accuracy                           0.55      8789
   macro avg       0.51      0.55      0.50      8789
weighted avg       0.62      0.55      0.56      8789



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
print('Random Forest Classifier')
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


Random Forest Classifier
              precision    recall  f1-score   support

          -1       0.80      0.17      0.28       784
           0       0.68      0.37      0.48      1582
           1       0.67      0.92      0.78      4514
           2       0.80      0.67      0.73      1909

    accuracy                           0.70      8789
   macro avg       0.74      0.53      0.57      8789
weighted avg       0.71      0.70      0.67      8789



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load the preprocessed data
data = pd.read_csv('cleaned_twitter_sentiment_data.csv',encoding='iso-8859-1')

# Tokenize the cleaned text
max_features = 10000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['cleaned_text'].values)
X = tokenizer.texts_to_sequences(data['cleaned_text'].values)
X = pad_sequences(X)

# Encode the sentiment labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['sentiment'])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the RNN model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=X.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))  # There are 4 sentiment classes: -1, 0, 1, and 2
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the RNN model
batch_size = 32
epochs = 10
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the RNN model
score, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print("Test score:", score)
print("Test accuracy:", accuracy)


Epoch 1/10
1099/1099 [==============================] - 895s 811ms/step - loss: 0.8019 - accuracy: 0.6808
Epoch 2/10
1099/1099 [==============================] - 895s 814ms/step - loss: 0.5506 - accuracy: 0.7859
Epoch 3/10
1099/1099 [==============================] - 907s 825ms/step - loss: 0.4366 - accuracy: 0.8329
Epoch 4/10
1099/1099 [==============================] - 886s 806ms/step - loss: 0.3496 - accuracy: 0.8685
Epoch 5/10
1099/1099 [==============================] - 888s 807ms/step - loss: 0.2803 - accuracy: 0.8969
Epoch 6/10
1099/1099 [==============================] - 885s 805ms/step - loss: 0.2219 - accuracy: 0.9166
Epoch 7/10
1099/1099 [==============================] - 889s 809ms/step - loss: 0.1846 - accuracy: 0.9331
Epoch 8/10
1099/1099 [==============================] - 887s 807ms/step - loss: 0.1535 - accuracy: 0.9421
Epoch 9/10
1099/1099 [==============================] - 882s 803ms/step - loss: 0.1262 - accuracy: 0.9532
Epoch 10/10
275/275 [=========================

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load the preprocessed dataset
data = pd.read_csv('cleaned_twitter_sentiment_data.csv',encoding='iso-8859-1')

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)
            embeddings[word] = vector
    return embeddings

glove_file_path = '/content/glove.6B.100d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)


# Convert tweets to GloVe embeddings
def tweet_to_glove_vector(tweet, embeddings):
    if not isinstance(tweet, str):
        return np.zeros(100)
    
    words = tweet.split()
    if not words:
        return np.zeros(100)
    
    word_vectors = [embeddings.get(word, np.zeros(100)) for word in words]
    return np.mean(word_vectors, axis=0)

data['glove_vector'] = data['cleaned_text'].apply(lambda x: tweet_to_glove_vector(x, glove_embeddings))

# Remove any rows with invalid GloVe vectors
data = data[data['glove_vector'].apply(lambda x: x.shape == (100,))]

# Save the GloVe values in the cleaned_twitter_sentiment_data.csv
data.to_csv('cleaned_twitter_sentiment_data.csv', index=False)


# Split the dataset into training and testing sets
X = np.vstack(data['glove_vector'].values)
y = data['sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression classifier
clf = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate the classifier
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          -1       0.50      0.15      0.23       784
           0       0.56      0.29      0.38      1582
           1       0.65      0.87      0.74      4514
           2       0.68      0.60      0.64      1909

    accuracy                           0.64      8789
   macro avg       0.59      0.48      0.50      8789
weighted avg       0.62      0.64      0.61      8789



In [ ]:
# Split the dataset into training and testing sets
X = np.vstack(data['glove_vector'].values)
y = data['sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train a Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
print('Random Forest Classifier')
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


Random Forest Classifier
              precision    recall  f1-score   support

          -1       0.95      0.10      0.18       784
           0       0.72      0.26      0.38      1582
           1       0.64      0.92      0.76      4514
           2       0.74      0.64      0.69      1909

    accuracy                           0.67      8789
   macro avg       0.76      0.48      0.50      8789
weighted avg       0.70      0.67      0.62      8789

